#Theoretical Questions


Computational Graphs And Gradients

When we call loss.backward() it calculates gradients and add them to.grad() attribute of leaf tensors. It does not overwrite the previous values.

Since Pytorch accumulates gradients they needs to be cleared otherwise the gradients will be added from previous batch to the current batch. This created massive problem because your gradient value depends on the average of previous terms. That is an important step.

Tensors View Vs Reshape

.view() is a operation that only works if the tensor is stored contiguously in memory. It simply changes how the underlying data is interpreted without moving it. .reshape() works if the tensor is non-contiguous, it will create a copy of the data in a new memory block to satisfy the shape request.

.reshape() is the safer fallback because it handles both contiguous and non-contiguous tensors automatically

Device Management

If you try to perform an operation like addition between a tensor on the CPU and one on cuda:0, PyTorch will raise a RuntimeError because Tensors must be on the same physical device memory to interact. PyTorch does not automatically move data between the host CPU and the device.

 When you move a model to the GPU using model.to('cuda'), only the existing parameters and buffers are moved.
 Any new tensors created from scratch inside the forward method do not automatically inherit the GPU device.
 You must manually assign them to the correct device, typically by using device=x.device to match the incoming data

#Programming Challenges

Question 4

In [2]:
import torch

def process_image_batch(images):

    return torch.where(images >= 0.5, 1.0, 0.0)


images = torch.rand(4, 28, 28)
processed = process_image_batch(images)

Question 5

In [4]:
def check_gradients():
    x = torch.tensor(4.0, requires_grad=True)

    y = x**3 + 2*x

    y.backward()

    calc_grad = x.grad.item()
    print(f"PyTorch Gradient: {calc_grad}")

    manual_grad = 3 * (4**2) + 2
    if calc_grad != manual_grad:
        raise ValueError(f"Mismatch! Expected {manual_grad}, got {calc_grad}")
    else:
        print("Gradients match perfectly.")

check_gradients()

PyTorch Gradient: 50.0
Gradients match perfectly.


Question 6

In [9]:
from torch.utils.data import Dataset

class NumberDataset(Dataset):
    def __init__(self, data_list):
        self.data = data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        num = self.data[idx]

        input_tensor = torch.tensor(float(num))
        target_tensor = torch.tensor(float(num * 2))
        return input_tensor, target_tensor

Question 7

In [13]:
import torch.nn as nn

class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Linear(10, 5)
        self.layer2 = nn.Linear(5, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

Question 8

In [14]:
def train_step(model, inputs, targets, optimizer, criterion):
    predictions = model(inputs)

    loss = criterion(predictions, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

model = SimpleClassifier()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

dummy_input = torch.randn(1, 10)
dummy_target = torch.tensor([[1.0]])

loss_val = train_step(model, dummy_input, dummy_target, optimizer, criterion)
print(f"Initial Loss: {loss_val}")

Initial Loss: 0.5009077191352844
